In [1]:
import base64
import pandas as pd
import requests as r
import ast

print("""
 _     _            ___  _____   __ ______      _        
| |   (_)           |  \/  |\ \ / / |  _  \    | |       
| |    _  __ _  __ _| .  . | \ V /  | | | |__ _| |_ __ _ 
| |   | |/ _` |/ _` | |\/| | /   \  | | | / _` | __/ _` |
| |___| | (_| | (_| | |  | |/ /^\ \ | |/ / (_| | || (_| |
\_____/_|\__, |\__,_\_|  |_/\/   \/ |___/ \__,_|\__\__,_|
          __/ |                                          
         |___/                                    
      """)


url = "https://ligamx.net/cancha/estadisticahistorica/1/"
base64_string = b'eyJpZERpdmlzaW9uIjoiMSIsImlkVGVtcG9yYWRhIjoiNzAiLCAiaWRUb3JuZW8iOiIxIn0='



 _     _            ___  _____   __ ______      _        
| |   (_)           |  \/  |\ \ / / |  _  \    | |       
| |    _  __ _  __ _| .  . | \ V /  | | | |__ _| |_ __ _ 
| |   | |/ _` |/ _` | |\/| | /   \  | | | / _` | __/ _` |
| |___| | (_| | (_| | |  | |/ /^\ \ | |/ / (_| | || (_| |
\_____/_|\__, |\__,_\_|  |_/\/   \/ |___/ \__,_|\__\__,_|
          __/ |                                          
         |___/                                    
      


In [2]:
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'} 

decoded = base64.decodebytes(base64_string)
dec = decoded.decode('UTF-8').replace("'", '"')

d = ast.literal_eval(dec)

n = int(d['idTemporada']) - 1

l = []

In [3]:
for i in range(11):
    for x in range(2,0,-1):
        l.append(str({'idDivision': '1', 'idTemporada': str(n), 'idTorneo': str(x)}).replace(' ', '').replace("idTorneo", " idTorneo").replace("'", '"').replace('"," ', '", "'))
    n-=1
    
encodedStrings = [str(base64.b64encode(bytes(i, 'utf-8')),'utf-8') for i in l]
urls = [(url + i) for i in encodedStrings]

dfs_list = []

In [4]:
def create_dfs(url):
    cont = r.get(url, headers = header).content
    t = pd.read_html(cont, attrs = {'class': 'default tbl_grals'})
    t[0].columns = t[0].columns.droplevel(level=0)
    df = t[0]
    columns=[('General','POS'),('General','Club'),('TOTAL','JJ'),('TOTAL','JG'), ('TOTAL','JE'),('TOTAL','JP'),('TOTAL','GF'),('TOTAL','GC'),('TOTAL','Dif'),('TOTAL','PTS'),
         ('LOCAL','JJ'),('LOCAL','JG'),('LOCAL','JE'),('LOCAL','JP'),('LOCAL','GF'),('LOCAL','GC'),('LOCAL','Dif'),('LOCAL','PTS'),
         ('VISITANTE','JJ'),('VISITANTE','JG'),('VISITANTE','JE'),('VISITANTE','JP'),('VISITANTE','GF'),('VISITANTE','GC'),('VISITANTE','Dif'),('VISITANTE','PTS')]
    df.columns=pd.MultiIndex.from_tuples(columns)
    dfs_list.append(df)

temporada = 2018
torneo = 2

#URL inicial insertada en input, generar DF inicial
create_dfs(url + str(base64_string))

In [5]:
try:
    for url in urls:
        create_dfs(url)
        print(f'[+] Temporada:{temporada}, Torneo:{torneo} --> Appended: ok!')
        torneo-=1
        if torneo == 0: 
            torneo = 2
            temporada -= 1
except ValueError as e: print(e)


[+] Temporada:2018, Torneo:2 --> Appended: ok!
[+] Temporada:2018, Torneo:1 --> Appended: ok!
[+] Temporada:2017, Torneo:2 --> Appended: ok!
[+] Temporada:2017, Torneo:1 --> Appended: ok!
[+] Temporada:2016, Torneo:2 --> Appended: ok!
[+] Temporada:2016, Torneo:1 --> Appended: ok!
[+] Temporada:2015, Torneo:2 --> Appended: ok!
[+] Temporada:2015, Torneo:1 --> Appended: ok!
[+] Temporada:2014, Torneo:2 --> Appended: ok!
[+] Temporada:2014, Torneo:1 --> Appended: ok!
[+] Temporada:2013, Torneo:2 --> Appended: ok!
[+] Temporada:2013, Torneo:1 --> Appended: ok!
[+] Temporada:2012, Torneo:2 --> Appended: ok!
[+] Temporada:2012, Torneo:1 --> Appended: ok!
No tables found


In [6]:
with pd.ExcelWriter('Estadisticos_LigaMX.xlsx') as writer:
    dfs_list[0].to_excel(writer, sheet_name=f'Temp 2019, Torneo 1')
    for df in dfs_list:
        df.to_excel(writer, sheet_name=f'Temp {temporada}, Torneo {torneo}')
        torneo-=1
        if torneo == 0: 
            torneo = 2
            temporada -= 1
print("[+] Excel Generado!")

[+] Excel Generado!


In [7]:
import os

os.listdir()

['liga_mx_data.ipynb',
 'Valores_Equipos_LigaMX.xlsx',
 '.gitkeep',
 'Estadisticos_LigaMX.xlsx',
 '.ipynb_checkpoints',
 'ligamx_equipos_valor.ipynb']

In [8]:
def sum_dfs(df_list):
    tables = []
    for table in df_list:
        table.columns = table.columns.droplevel(level=0)
        table.sort_values('Club', inplace = True)
        tables.append(table)
        result = pd.concat(tables).groupby(['Club']).sum()          
    return tables

    

In [9]:
sum_dfs(dfs_list)

[    POS                        Club  JJ  JG  JE  JP  GF  GC  Dif  PTS  ...  \
 4     5                    AmÃ©rica  17   7   7   3  27  22    5   28  ...   
 11   12                       Atlas  17   6   3   8  19  24   -5   21  ...   
 13   14  Club AtlÃ©tico de San Luis  16   6   2   8  21  26   -5   20  ...   
 12   13                   Cruz Azul  16   4   8   4  21  20    1   20  ...   
 15   16                  FC JuÃ¡rez  16   5   2   9  15  24   -9   17  ...   
 14   15                 Guadalajara  16   5   4   7  22  25   -3   19  ...   
 5     6                       LeÃ³n  16   7   6   3  32  22   10   27  ...   
 6     7                  M. Morelia  16   7   3   6  27  21    6   24  ...   
 10   11                   Monterrey  16   6   3   7  21  23   -2   21  ...   
 1     2                      Necaxa  16   8   4   4  31  20   11   28  ...   
 8     9                       PUMAS  16   6   4   6  20  17    3   22  ...   
 9    10                     Pachuca  16   6   3   7

In [10]:
result = pd.concat(dfs_list)

In [11]:
final_table = result.groupby(['Club']).sum()

In [12]:
final_table

,POS,JJ,JG,JE,JP,GF,GC,Dif,PTS,JJ,...,Dif,PTS,JJ,JG,JE,JP,GF,GC,Dif,PTS
Club,,,,,,,,,,,,,,,,,,,,,
AmÃ©rica,56,255,124,71,60,406,273,133,443,128,...,96,244,127,52,43,32,188,151,37,199
Atlante,66,68,17,12,39,78,130,-52,63,34,...,-9,40,34,7,2,25,30,73,-43,23
Atlas,171,255,78,72,105,282,348,-66,306,128,...,-6,168,127,35,33,59,138,198,-60,138
Club AtlÃ©tico de San Luis,14,16,6,2,8,21,26,-5,20,8,...,-6,5,8,5,0,3,14,13,1,15
Cruz Azul,122,254,99,86,69,341,280,61,383,127,...,65,217,127,39,49,39,157,161,-4,166
Dorados,34,34,7,8,19,31,61,-30,29,17,...,-2,22,17,1,4,12,13,41,-28,7
FC JuÃ¡rez,16,16,5,2,9,15,24,-9,17,8,...,4,13,8,1,1,6,4,17,-13,4
Guadalajara,172,254,76,80,98,279,317,-38,308,126,...,-7,160,128,36,40,52,140,171,-31,148
Jaguares,119,170,52,48,70,212,272,-60,204,85,...,10,136,85,15,23,47,87,157,-70,68


In [13]:
final_table = final_table.drop(columns = ['POS'])

In [14]:
columns=[('TOTAL','JJ'),('TOTAL','JG'), ('TOTAL','JE'),('TOTAL','JP'),('TOTAL','GF'),('TOTAL','GC'),('TOTAL','Dif'),('TOTAL','PTS'),
         ('LOCAL','JJ'),('LOCAL','JG'),('LOCAL','JE'),('LOCAL','JP'),('LOCAL','GF'),('LOCAL','GC'),('LOCAL','Dif'),('LOCAL','PTS'),
         ('VISITANTE','JJ'),('VISITANTE','JG'),('VISITANTE','JE'),('VISITANTE','JP'),('VISITANTE','GF'),('VISITANTE','GC'),('VISITANTE','Dif'),('VISITANTE','PTS')]

final_table.columns=pd.MultiIndex.from_tuples(columns)

### Vemos que del 2012 a la fecha el equipo que ha generado mas puntos totales es el America, seguido de Tigres
### Podemos decir que el top 5 de los equipos mexicanos son del norte (MTY) y de la Ciudad de Mexico mas Toluca

#### *Es necesario revisar el caso de los equipos con menos de 250 juegos (promediar), pues no han estado en la misma cantidad de torneos

In [15]:
final_sorted_by_pts = final_table.sort_values([('TOTAL', 'PTS')], ascending=False)
final_sorted_by_pts

TOTAL                                   LOCAL      \
                              JJ   JG  JE   JP   GF   GC  Dif  PTS    JJ  JG   
Club                                                                           
AmÃ©rica                     255  124  71   60  406  273  133  443   128  72   
UANL                         254  114  81   59  380  240  140  423   127  67   
Toluca                       254  112  65   77  357  290   67  401   127  70   
Monterrey                    254  111  66   77  398  322   76  399   127  71   
Cruz Azul                    254   99  86   69  341  280   61  383   127  60   
LeÃ³n                        254  108  56   90  401  338   63  380   127  62   
Santos Laguna                254   98  82   74  373  333   40  376   127  66   
Tijuana                      254   97  67   90  326  331   -5  358   126  65   
Pachuca                      254   95  68   91  366  331   35  353   127  64   
PUMAS                        254   91  69   94  332  332    0  342   127  53   
M. Morelia                   254   89  71   94  345  355  -10  338   127  50   
Guadalajara                  254   76  80   98  279  317  -38  308   126  40   
Atlas                        255   78  72  105  282  348  -66  306   128  43   
QuerÃ©taro                   254   80  64  110  290  348  -58  304   127  48   
Puebla                       204   55  64   85  228  280  -52  229   102  36   
Jaguares                     170   52  48   70  212  272  -60  204    85  37   
Veracruz                     220   49  60  111  207  365 -158  201   110  33   
Necaxa                       118   40  44   34  162  142   20  164    60  20   
Lobos                         68   20  11   37   82  123  -41   71    34  12   
Atlante                       68   17  12   39   78  130  -52   63    34  10   
Puebla F.C.                   50   14  16   20   56   79  -23   58    25   7   
U. de G.                      34    8  11   15   23   35  -12   35    17   4   
San Luis                      34    7  10   17   33   49  -16   31    17   3   
Dorados                       34    7   8   19   31   61  -30   29    17   6   
Club AtlÃ©tico de San Luis    16    6   2    8   21   26   -5   20     8   1   
FC JuÃ¡rez                    16    5   2    9   15   24   -9   17     8   4   

                            ...           VISITANTE                        \
                            ...  Dif  PTS        JJ  JG  JE  JP   GF   GC   
Club                        ...                                             
AmÃ©rica                    ...   96  244       127  52  43  32  188  151   
UANL                        ...  110  240       127  47  42  38  153  123   
Toluca                      ...   92  239       127  42  36  49  143  168   
Monterrey                   ...  115  254       127  40  25  62  158  197   
Cruz Azul                   ...   65  217       127  39  49  39  157  161   
LeÃ³n                       ...   77  215       127  46  27  54  165  179   
Santos Laguna               ...   80  232       127  32  48  47  149  189   
Tijuana                     ...   73  228       128  32  34  62  131  209   
Pachuca                     ...   82  224       127  31  36  60  151  198   
PUMAS                       ...   45  197       127  38  31  58  141  186   
M. Morelia                  ...   24  183       127  39  38  50  158  192   
Guadalajara                 ...   -7  160       128  36  40  52  140  171   
Atlas                       ...   -6  168       127  35  33  59  138  198   
QuerÃ©taro                  ...   -6  176       127  32  32  63  141  193   
Puebla                      ...    6  138       102  19  34  49   96  154   
Jaguares                    ...   10  136        85  15  23  47   87  157   
Veracruz                    ...  -30  129       110  16  30  64   84  212   
Necaxa                      ...   20   89        58  20  15  23   76   76   
Lobos                       ...   -3   42        34   8   5  21   35   73   
Atlante     

In [16]:
top_ten = final_sorted_by_pts[:10]
top_ten

TOTAL                                  LOCAL      ...            \
                 JJ   JG  JE  JP   GF   GC  Dif  PTS    JJ  JG  ...  Dif  PTS   
Club                                                            ...             
AmÃ©rica        255  124  71  60  406  273  133  443   128  72  ...   96  244   
UANL            254  114  81  59  380  240  140  423   127  67  ...  110  240   
Toluca          254  112  65  77  357  290   67  401   127  70  ...   92  239   
Monterrey       254  111  66  77  398  322   76  399   127  71  ...  115  254   
Cruz Azul       254   99  86  69  341  280   61  383   127  60  ...   65  217   
LeÃ³n           254  108  56  90  401  338   63  380   127  62  ...   77  215   
Santos Laguna   254   98  82  74  373  333   40  376   127  66  ...   80  232   
Tijuana         254   97  67  90  326  331   -5  358   126  65  ...   73  228   
Pachuca         254   95  68  91  366  331   35  353   127  64  ...   82  224   
PUMAS           254   91  69  94  332  332    0  342   127  53  ...   45  197   

              VISITANTE                                 
                     JJ  JG  JE  JP   GF   GC Dif  PTS  
Club                                                    
AmÃ©rica            127  52  43  32  188  151  37  199  
UANL                127  47  42  38  153  123  30  183  
Toluca              127  42  36  49  143  168 -25  162  
Monterrey           127  40  25  62  158  197 -39  145  
Cruz Azul           127  39  49  39  157  161  -4  166  
LeÃ³n               127  46  27  54  165  179 -14  165  
Santos Laguna       127  32  48  47  149  189 -40  144  
Tijuana             128  32  34  62  131  209 -78  130  
Pachuca             127  31  36  60  151  198 -47  129  
PUMAS               127  38  31  58  141  186 -45  145  

[10 rows x 24 columns]

# Los Mejores 10

#### Revisando el top 10 en resultados generales, vemos que el America es quien mas goles a favor tiene sin embargo tigres es la mejor defensiva y mejor diferencia de goles vemos el caso de Leon por ejemplo, quien es el segundo mejor goleador pero es 6 lugar general y tiene considerables goles en contra
#### Finalmente vemos el caso de Tijuana quien a pesar de tener dif de goles en contra, se ubica mejor que pachuca quien tiene 40 goles mas a favor

In [17]:
top_total = final_sorted_by_pts.sort_values([('TOTAL', 'PTS')], ascending=False)
top_ten_total = top_total[:10]
top_ten_total['TOTAL']

,JJ,JG,JE,JP,GF,GC,Dif,PTS
Club,,,,,,,,
AmÃ©rica,255,124,71,60,406,273,133,443
UANL,254,114,81,59,380,240,140,423
Toluca,254,112,65,77,357,290,67,401
Monterrey,254,111,66,77,398,322,76,399
Cruz Azul,254,99,86,69,341,280,61,383
LeÃ³n,254,108,56,90,401,338,63,380
Santos Laguna,254,98,82,74,373,333,40,376
Tijuana,254,97,67,90,326,331,-5,358
Pachuca,254,95,68,91,366,331,35,353


# Los Mejores 10 Locales

#### Podemos ver que el mejor jugador de local es Rayados  y mejor goleador 
#### El America es el segundo mejor jugadando de local
#### Tigres  es tercero y tiene la mejor defensiva jugando de local y es segundo mejor goleador
#### Por lo que los equipos de MTY jugando de local tienen la mejor ofensiva y defensiva

In [18]:
top_local = final_sorted_by_pts.sort_values([('LOCAL', 'PTS')], ascending=False)
top_local_ten = top_local[:10]

In [19]:
top_local_ten['LOCAL']

,JJ,JG,JE,JP,GF,GC,Dif,PTS
Club,,,,,,,,
Monterrey,127,71,41,15,240,125,115,254
AmÃ©rica,128,72,28,28,218,122,96,244
UANL,127,67,39,21,227,117,110,240
Toluca,127,70,29,28,214,122,92,239
Santos Laguna,127,66,34,27,224,144,80,232
Tijuana,126,65,33,28,195,122,73,228
Pachuca,127,64,32,31,215,133,82,224
Cruz Azul,127,60,37,30,184,119,65,217
LeÃ³n,127,62,29,36,236,159,77,215


# Los Mejores 10 Visitantes
#### Como visitantes America es el mejor, seguido de Tigres
#### Tigres es el mejor defensivo de visitante
#### Tanto Tigres como America son los unicos quienes mantienen goles a favor jugando de visitante
#### Podemos decir que generalmente quien juega de local hace mas goles excepto cuando el rival es Tigres o America
#### Ademas son los unicos que equipos que tienen mas ganes que empates o derrotas como visitantes

In [20]:
top_vis = final_sorted_by_pts.sort_values([('VISITANTE', 'PTS')], ascending=False)
top_vis_ten = top_vis[:10]

In [21]:
top_vis_ten
top_vis_ten['VISITANTE']

,JJ,JG,JE,JP,GF,GC,Dif,PTS
Club,,,,,,,,
AmÃ©rica,127,52,43,32,188,151,37,199
UANL,127,47,42,38,153,123,30,183
Cruz Azul,127,39,49,39,157,161,-4,166
LeÃ³n,127,46,27,54,165,179,-14,165
Toluca,127,42,36,49,143,168,-25,162
M. Morelia,127,39,38,50,158,192,-34,155
Guadalajara,128,36,40,52,140,171,-31,148
Monterrey,127,40,25,62,158,197,-39,145
PUMAS,127,38,31,58,141,186,-45,145
